In [54]:
import sqlalchemy as sql
import pandas as pd
import glob
from os import path, sep
import numpy as np
import matplotlib.pyplot as plt
from numpy.fft import fft, fftshift
from scipy.signal import oaconvolve
from scipy.signal import remez
import time
import decimate as d
from importlib import reload
reload(d);

In [46]:
time.perf_counter()

15380.98463282

In [28]:
f3 = remez(4000,[0., 0.5, 1., 250.],[1.,0.],fs=500)


In [2]:
server = "thesis.ca6j6heoraog.eu-central-1.rds.amazonaws.com"
engine = sql.create_engine(f"mysql+pymysql://admin:FphvsYQek4@{server}/thesis_db")

In [3]:
con = engine.connect()

```SQL
create table interpolation_types (
	id int(11) NOT NULL PRIMARY KEY AUTO_INCREMENT,
    interp_algo varchar(45),
    algo_version varchar(20),
    notebook_name varchar(45),
    sampling_rate float(6,4)
    )
```

In [5]:

stmnt ="""
INSERT INTO interpolation_series
values 
    (NULL, "legacy", "", NULL, 0.5),
    (Null, "piecewise", "ec31c78", "interpolate and upload", 0.25)

"""

con.execute(stmnt)

In [61]:
stmnt = """
SELECT DISTINCT series_type_id, group_id, participant_id
FROM raw_interval_data
"""

raw_series = con.execute(stmnt)

In [62]:
raw_series

In [63]:
get_series = sql.text("""
SELECT data_time
FROM raw_interval_data
WHERE series_type_id = :sti
AND group_id = :gi
AND participant_id = :pi""")

In [65]:
tref = time.perf_counter()
for series_type_id, group_id, participant_id in raw_series:
    print(series_type_id, group_id, participant_id, time.perf_counter()-tref ) 
    q = con.execute(get_series, sti=series_type_id, gi= group_id, pi=participant_id)
    t = np.array(q.fetchall()).astype(float).squeeze()
    x = t[:-1] # don't use the point which marks the end of the experiment
    y = np.diff(t)
    x_pw = np.arange(0,t[-1],0.5)
    y_pw = d.pw_constant_convolve(x, y, t[-1], f3, 4, 3, x_pw)
    nx = len(x_pw)
    interpolated_data = pd.DataFrame({
        'series_type_id': [series_type_id]*nx,
        'interpolation_series_id': [2]*nx,
        'group_id': [group_id]*nx,
        'participant_id': [participant_id]*nx,
        'data_ordinal': range(nx),
        'data_time': x_pw,
        'data_value': np.around(y_pw,5)
    })
    interpolated_data.to_sql(name='interpolated_interval_data', con=con, if_exists='append', index=False, method='multi')   


1 1001 101 0.00018914399697678164
1 1001 102 0.423030709996965
1 1001 103 0.8007449059987266
1 1003 108 1.1928054709969729
1 1003 109 1.557995489998575
1 1004 110 1.9358573049976258
1 1004 111 2.309690187998058
1 1004 112 2.6630400869980804
1 1005 113 3.114963241998339
1 1005 114 3.44203883499722
1 1005 115 3.8063801689968386
1 1006 116 4.168765847996838
1 1006 117 4.533413227996789
1 1006 118 4.906866888999502
1 1007 119 5.270275912000216
1 1007 120 5.7634666499980085
1 1007 121 6.136644504000287
1 1008 122 6.516148481998243
1 1008 123 6.884609577999072
1 1008 124 7.27975488499942
1 1009 125 7.648246182998264
1 1009 126 8.022839062999992
1 1009 127 8.39118428599977
1 1010 128 8.761580167996726
1 1010 129 9.244082277997222
1 1010 130 9.617343912999786
1 1011 131 9.99492048999673
1 1011 132 10.385021907997725
1 1011 133 10.77896567299831
1 1012 134 11.105033086998446
1 1012 135 11.496885063999798
1 1012 136 11.956941713997367
1 1013 137 12.316230648997589
1 1013 138 12.679508082997927
1

3 1003 108 104.9070675169969
3 1003 109 105.27320595399942
3 1004 110 105.63834497399876
3 1004 111 106.04589426199891
3 1004 112 106.40542592899874
3 1005 113 106.8856818489985
3 1005 114 107.25683408999976
3 1005 115 107.63041505199726
3 1006 116 107.98777835599685
3 1006 117 108.37905160399896
3 1006 118 108.74097501099823
3 1007 119 109.12166951999825
3 1007 120 109.50738932099921
3 1007 121 109.87699664299726
3 1008 122 110.31331538799714
3 1008 123 110.68850933799695
3 1008 124 111.06864795699948
3 1009 125 111.45127878799758
3 1009 126 111.82864635499936
3 1009 127 112.2107505219974
3 1010 128 112.57068493499901
3 1010 129 113.03777335999985
3 1010 130 113.34879596999963
3 1011 131 113.67648363199987
3 1011 132 114.03497878399867
3 1011 133 114.39456506399802
3 1012 134 114.75779683200017
3 1012 135 115.1173150709983
3 1012 136 115.58612408399858
3 1013 137 115.91819786799897
3 1013 138 116.28589705599734
3 1013 139 116.67193831999975
3 1014 140 117.04248418099814
3 1014 141 117

In [55]:
y_pw = d.pw_constant_convolve(x, y, t[-1], f3, 4, 3, x_pw)
nx = len(x_pw)
interpolated_data = pd.DataFrame({
    'series_type_id': [series_type_id]*nx,
    'group_id': [group_id]*nx,
    'participant_id': [participant_id]*nx,
    'data_ordinal': range(nx),
    'data_time': x_pw,
    'data_value': np.around(y_pw,5)
})
interpolated_data.to_sql(name='interpolated_interval_data', con=con, if_exists='append', index=False, method='multi')   


In [21]:
df

,series_type_id,group_id,participant_id,data_ordinal,data_time,data_time_diff
0,1,1035,206,0,0.39600,None
1,1,1035,206,1,1.05600,0.66000
2,1,1035,206,2,1.71600,0.66000
3,1,1035,206,3,2.34400,0.62800
4,1,1035,206,4,2.96000,0.61600
...,...,...,...,...,...,...
408,1,1035,206,408,312.04000,0.77400
409,1,1035,206,409,312.84800,0.80800
410,1,1035,206,410,313.65800,0.81000
411,1,1035,206,411,314.47800,0.82000


In [49]:
(t[-1], f3, 4, 3, x_pw)

(318.004,
 array([ 1.61205331e-04, -2.39064541e-06, -2.37369060e-06, ...,
        -2.37369060e-06, -2.39064541e-06,  1.61205331e-04]),
 4,
 3,
 array([  0. ,   0.5,   1. ,   1.5,   2. ,   2.5,   3. ,   3.5,   4. ,
          4.5,   5. ,   5.5,   6. ,   6.5,   7. ,   7.5,   8. ,   8.5,
          9. ,   9.5,  10. ,  10.5,  11. ,  11.5,  12. ,  12.5,  13. ,
         13.5,  14. ,  14.5,  15. ,  15.5,  16. ,  16.5,  17. ,  17.5,
         18. ,  18.5,  19. ,  19.5,  20. ,  20.5,  21. ,  21.5,  22. ,
         22.5,  23. ,  23.5,  24. ,  24.5,  25. ,  25.5,  26. ,  26.5,
         27. ,  27.5,  28. ,  28.5,  29. ,  29.5,  30. ,  30.5,  31. ,
         31.5,  32. ,  32.5,  33. ,  33.5,  34. ,  34.5,  35. ,  35.5,
         36. ,  36.5,  37. ,  37.5,  38. ,  38.5,  39. ,  39.5,  40. ,
         40.5,  41. ,  41.5,  42. ,  42.5,  43. ,  43.5,  44. ,  44.5,
         45. ,  45.5,  46. ,  46.5,  47. ,  47.5,  48. ,  48.5,  49. ,
         49.5,  50. ,  50.5,  51. ,  51.5,  52. ,  52.5,  53. ,  53.5,
     

In [52]:
x

array([7.00000e-02, 8.26000e-01, 1.55800e+00, 2.28200e+00, 3.08200e+00,
       3.86400e+00, 4.56800e+00, 5.27400e+00, 6.04400e+00, 6.85000e+00,
       7.65400e+00, 8.48000e+00, 9.35200e+00, 1.02380e+01, 1.10500e+01,
       1.19000e+01, 1.27320e+01, 1.34840e+01, 1.42900e+01, 1.51140e+01,
       1.59640e+01, 1.67780e+01, 1.76540e+01, 1.85560e+01, 1.94360e+01,
       2.03240e+01, 2.12380e+01, 2.21220e+01, 2.29740e+01, 2.38540e+01,
       2.47480e+01, 2.55840e+01, 2.64540e+01, 2.73440e+01, 2.81940e+01,
       2.90680e+01, 2.99460e+01, 3.07680e+01, 3.16040e+01, 3.24420e+01,
       3.32720e+01, 3.40480e+01, 3.48200e+01, 3.56060e+01, 3.63720e+01,
       3.71620e+01, 3.79900e+01, 3.88060e+01, 3.95700e+01, 4.03640e+01,
       4.11880e+01, 4.19360e+01, 4.26780e+01, 4.34420e+01, 4.42480e+01,
       4.50480e+01, 4.58520e+01, 4.65980e+01, 4.74000e+01, 4.81920e+01,
       4.90260e+01, 4.98320e+01, 5.06680e+01, 5.15120e+01, 5.23760e+01,
       5.32680e+01, 5.41060e+01, 5.49340e+01, 5.57680e+01, 5.658

In [53]:
y

array([  0.756,   0.732,   0.724,   0.8  ,   0.782,   0.704,   0.706,
         0.77 ,   0.806,   0.804,   0.826,   0.872,   0.886,   0.812,
         0.85 ,   0.832,   0.752,   0.806,   0.824,   0.85 ,   0.814,
         0.876,   0.902,   0.88 ,   0.888,   0.914,   0.884,   0.852,
         0.88 ,   0.894,   0.836,   0.87 ,   0.89 ,   0.85 ,   0.874,
         0.878,   0.822,   0.836,   0.838,   0.83 ,   0.776,   0.772,
         0.786,   0.766,   0.79 ,   0.828,   0.816,   0.764,   0.794,
         0.824,   0.748,   0.742,   0.764,   0.806,   0.8  ,   0.804,
         0.746,   0.802,   0.792,   0.834,   0.806,   0.836,   0.844,
         0.864,   0.892,   0.838,   0.828,   0.834,   0.816,   0.786,
         0.82 ,   0.854,   0.82 ,   0.812,   0.844,   0.804,   0.774,
         0.816,   0.864,   0.916,   0.87 ,   0.876,   0.88 ,   0.86 ,
         0.836,   0.858,   0.882,   0.852,   0.836,   0.86 ,   0.876,
         0.9  ,   0.814,   0.774,   0.784,   0.792,   0.76 ,   0.76 ,
         0.786,   0.